In [ ]:
import requests
from bs4 import BeautifulSoup

# 1. Obtener la URL
url = "https://books.toscrape.com/"
respuesta_obtenida = requests.get(url)
html_obtenido = respuesta_obtenida.text  # HTML como string

# 2. Parsear ese HTML con BeautifulSoup
soup = BeautifulSoup(html_obtenido, 'html.parser')  # <- ¡ESTO FALTABA!

# 3. Buscar el primer título <h3>
primer_h3 = soup.find('h3')

# 4. Mostrar el contenido
print(primer_h3)         # Muestra todo el tag <h3>...</h3>
print(primer_h3.text)    # Muestra solo el texto (título del libro)
print(soup.h3.text)      # Equivalente

# 5. Trayendo todos los <h3>

h3_todos = soup.find_all('h3')
print(h3_todos) #lista con todos los elementos de h3

# 6. Iteracion de todos los elementos poniendo solo texto 
contador = 0
for unoporuno in h3_todos:
    contador+=1

    print(contador, unoporuno.text)
    






In [4]:
import requests
from bs4 import BeautifulSoup
libros = None


base_url = "https://books.toscrape.com/catalogue/page-{}.html"


#funcion para recorrer todas las paginas 

for page in range(1, 2):  # Hay 50 páginas en total
    url = base_url.format(page)
    respuesta = requests.get(url)
    soup = BeautifulSoup(respuesta.text, 'html.parser')

    # Aquí haces scraping de los libros de esa página
    libros = soup.find_all('article', class_='product_pod')
    print(f"📄 Página {page} -> {len(libros)} libros encontrados")

# recorrer_paginas()


#   

for libro in libros:
    titulo = libro.h3.a['title']
    precio = libro.find('p', class_='price_color').text
    disponibilidad = libro.find('p', class_='instock availability').text.strip()

    # Enlace al detalle del libro (relativo -> convertir a absoluto)
    enlace_relativo = libro.h3.a['href']
    enlace_absoluto = "https://books.toscrape.com/catalogue/" + enlace_relativo

    print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 🔗 {enlace_absoluto}")
    
    
    
detalle = requests.get(enlace_absoluto)
soup_detalle = BeautifulSoup(detalle.text, 'html.parser')

# Categoría: está en un breadcrumb (<ul class="breadcrumb">)
categoria = soup_detalle.find('ul', class_='breadcrumb').find_all('a')[-1].text

print(f"📚 {titulo} | 💵 {precio} | 📦 {disponibilidad} | 📂 {categoria}")

    
    





📄 Página 1 -> 20 libros encontrados
📚 A Light in the Attic | 💵 Â£51.77 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html
📚 Tipping the Velvet | 💵 Â£53.74 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html
📚 Soumission | 💵 Â£50.10 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/soumission_998/index.html
📚 Sharp Objects | 💵 Â£47.82 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sharp-objects_997/index.html
📚 Sapiens: A Brief History of Humankind | 💵 Â£54.23 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html
📚 The Requiem Red | 💵 Â£22.65 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-requiem-red_995/index.html
📚 The Dirty Little Secrets of Getting Your Dream Job | 💵 Â£33.34 | 📦 In stock | 🔗 https://books.toscrape.com/catalogue/the-dirty-little-secrets-of-getting-your-dream-job_994/index.html
📚 The Coming Woman: A Novel Based on the Li